# Reformat data info for dataloaders

In [1]:
import sys
sys.path.append('/media/rene/LLR/lib/python3.6/site-packages')

import os
import sys
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


PATH = Path('/media/rene/data/art')

In [16]:
info = pd.read_csv(Path(PATH/'all_data_info.csv'))
display(info.head())

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [17]:
display(info['style'].value_counts())

Impressionism                   10643
Realism                         10523
Romanticism                      9285
Expressionism                    7013
Post-Impressionism               5778
Art Nouveau (Modern)             4899
Baroque                          4400
Surrealism                       4167
Symbolism                        3476
Rococo                           2733
Northern Renaissance             2379
Naïve Art (Primitivism)          2295
Abstract Expressionism           2074
Neoclassicism                    2038
Cubism                           1747
Ukiyo-e                          1426
Early Renaissance                1351
Mannerism (Late Renaissance)     1342
High Renaissance                 1314
Art Informel                     1267
Magic Realism                    1002
Abstract Art                      979
Academicism                       972
Color Field Painting              910
Pop Art                           791
Fauvism                           731
Lyrical Abst

In [18]:
styles = {
    'Impressionism': 0,
    'Realism': 1,
    'Romanticism': 2,
    'Expressionism': 3,
    'Post-Impressionism': 4,
    'Art Nouveau (Modern)': 5,
    'Surrealism': 6,
    'Naïve Art (Primitivism)': 7,
    'Abstract Expressionism': 8,
    'Cubism': 9,
    'Ukiyo-e': 10
}

new_info = info[info['style'].isin(list(styles.keys()))]

new_info["style"] = new_info["style"].map(styles)

new_info['path'] = new_info['new_filename']
new_info['label'] = new_info['style']

new_info = new_info[['path','label']]
print(len(new_info))

59850


/home/rene/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/rene/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/rene/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

In [20]:
drop_rows = []
for index, row in new_info.iterrows():
    try:
        loc = str(PATH) +'/train/'+ row['path']
        img = Image.open(loc)
    except Exception as e:
#         print(e)
        drop_rows.append(index)
        
print('len(drop_rows)', len(drop_rows))
new_info = new_info.drop(drop_rows)
print('len(new_info)', len(new_info))
display(new_info.head())

/media/rene/LLR/lib/python3.6/site-packages/PIL/Image.py:2575: DecompressionBombWarning: Image size (94435468 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/media/rene/LLR/lib/python3.6/site-packages/PIL/Image.py:2575: DecompressionBombWarning: Image size (145486286 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/media/rene/LLR/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/media/rene/LLR/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 787528 bytes but only got 5533. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/media/rene/LLR/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF da

len(drop_rows) 13940
len(new_info) 45910


,path,label
10,53180.jpg,10
11,70944.jpg,6
13,48138.jpg,10
14,99442.jpg,2
17,23508.jpg,10


In [22]:
files = {}
# files['val'] = new_info.groupby('style', group_keys=False).apply(lambda x: x.sample(len(x)*.1))
msk = np.random.rand(len(new_info)) < 0.9
files['train'] = new_info[msk]
files['val'] = new_info[~msk]

print(files['val']['label'].value_counts())
print(files['train']['label'].value_counts())

0     819
1     768
2     699
3     532
4     458
5     378
6     307
7     183
8     150
10    126
9     122
Name: label, dtype: int64
0     7400
1     7344
2     6342
3     4792
4     4069
5     3401
6     2826
7     1593
8     1396
9     1194
10    1011
Name: label, dtype: int64


In [23]:
with open(str(PATH)+'/files_dict.pkl', 'wb') as f:
    pickle.dump(files, f, pickle.HIGHEST_PROTOCOL)

## Resize images and save

In [2]:
files_dict_loc = str(PATH)+'/files_dict.pkl'
with open(files_dict_loc, 'rb') as f:
    files_dict = pickle.load(f)
    
size = 64

for folder, files_df in files_dict.items():
    for index, row in tqdm(files_df.iterrows(), total=files_df.shape[0]):
        try:
            loc = str(PATH) +'/train/'+ row['path']
#             for size in [64]:
            img = Image.open(loc).convert('RGB').resize((size, size))
            img.save(str(PATH) +'/'+str(folder)+'_'+str(size)+'/'+ row['path'],"PNG", compress_level=2)
        except Exception as e:
            print(e)

  0%|          | 11/41368 [00:11<11:22:39,  1.01it/s]

KeyboardInterrupt: 

## Clean Again

In [9]:
files_dict_loc = str(PATH)+'/files_dict.pkl'
with open(files_dict_loc, 'rb') as f:
    files_dict = pickle.load(f)
    
for ttv, files_df in files_dict.items():
    print('folder len(new_info)', ttv, len(files_df))
    drop_rows = []
    for index, row in files_dict[ttv].iterrows():
        try:
            loc = str(PATH) +'/train_64/'+ row['path']
            img = Image.open(loc)
        except Exception as e:
            drop_rows.append(index)

    print('len(drop_rows)', len(drop_rows))
    files_df = files_dict[ttv].drop(drop_rows)
    print('len(files_dict)', len(files_dict[ttv]))
    display(files_dict[ttv].head())

folder len(new_info) train 41368
len(drop_rows) 0
len(files_dict) 41368


,path,label
10,53180.jpg,10
11,70944.jpg,6
14,99442.jpg,2
17,23508.jpg,10
21,47145.jpg,10


folder len(new_info) val 4542
len(drop_rows) 0
len(files_dict) 4542


,path,label
13,48138.jpg,10
19,27254.jpg,10
43,4147.jpg,9
93,73181.jpg,5
193,22404.jpg,2


In [ ]:
with open(str(PATH)+'/files_dict.pkl', 'wb') as f:
    pickle.dump(files, f, pickle.HIGHEST_PROTOCOL)